In [16]:
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime

In [6]:
from sklearn.decomposition import PCA

In [2]:
data=pd.read_csv("clean_data_for_analysis.csv")

In [3]:
data

,Title,Type,Release,Runtime,Genre,Nominations,Metascore,imdbRating,imdbVotes,imdbID,Budget,Gross,Platform
0,Insidious: The Last Key,Movie,2018-01-05,6180.0,Horror,NaN,49.0,5.7,63102,tt5726086,10000000.0,1.678856e+08,others
1,Stratton,Movie,2018-01-05,5700.0,Action,NaN,26.0,4.8,5405,tt3567666,NaN,2.572120e+05,others
2,Sweet Country,Movie,2018-04-06,6780.0,Adventure,NaN,88.0,6.9,8855,tt6958212,NaN,1.849718e+06,others
3,The Commuter,Movie,2018-01-12,6240.0,Action,NaN,56.0,6.3,118954,tt1590193,30000000.0,1.199424e+08,others
4,Freak Show,Movie,2018-01-12,5460.0,Comedy,NaN,54.0,6.5,4376,tt5089534,NaN,2.065700e+04,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,Avatar: The Way of Water,Movie,2022-12-16,11520.0,Action,3.0,67.0,7.7,355332,tt1630029,350000000.0,2.315057e+09,others
1149,Puss in Boots: The Last Wish,Movie,2022-12-21,6120.0,Animation,1.0,73.0,7.9,117294,tt3915174,90000000.0,4.800034e+08,others
1150,Wildcat,Movie,2022-12-30,6360.0,Documentary,NaN,71.0,7.8,2605,tt13622204,NaN,NaN,others
1151,Babylon,Movie,2022-12-23,11340.0,Comedy,3.0,60.0,7.2,107214,tt10640346,78000000.0,6.337762e+07,others


In [4]:
features=data[['Release', 'Runtime', 'Genre', 'Budget', 'Platform']]

In [5]:
outcome=data['Gross']

In [7]:
features


,Release,Runtime,Genre,Budget,Platform
0,2018-01-05,6180.0,Horror,10000000.0,others
1,2018-01-05,5700.0,Action,NaN,others
2,2018-04-06,6780.0,Adventure,NaN,others
3,2018-01-12,6240.0,Action,30000000.0,others
4,2018-01-12,5460.0,Comedy,NaN,others
...,...,...,...,...,...
1148,2022-12-16,11520.0,Action,350000000.0,others
1149,2022-12-21,6120.0,Animation,90000000.0,others
1150,2022-12-30,6360.0,Documentary,NaN,others
1151,2022-12-23,11340.0,Comedy,78000000.0,others


In [8]:
# Create a DataFrame with a datetime column
df = pd.DataFrame({'date_column': pd.to_datetime(['2023-04-15 12:34:56', '2023-04-16 01:23:45', '2023-04-17 09:00:00'])})

# Convert datetime to float representing days since a reference date with fractional part for hours, minutes, and seconds
df['date_column_float'] = (df['date_column'] - pd.Timestamp('1970-01-01')) / pd.Timedelta('1D')

print(df)

          date_column  date_column_float
0 2023-04-15 12:34:56       19462.524259
1 2023-04-16 01:23:45       19463.058160
2 2023-04-17 09:00:00       19464.375000


In [12]:
test=features['Release'][0]
test

'2018-01-05'

In [17]:
dt_obj = datetime.strptime(test, '%Y-%m-%d')

In [18]:
dt_obj

datetime.datetime(2018, 1, 5, 0, 0)

In [19]:
unix_datetime = int(dt_obj.timestamp())

In [20]:
unix_datetime

1515139200

In [21]:
convert_to_unix = lambda x: int(datetime.strptime(x, '%Y-%m-%d').timestamp())

features['Release']=features['Release'].apply(convert_to_unix)

/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_8289/2456918480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Release']=features['Release'].apply(convert_to_unix)


In [23]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
features

,Release,Runtime,Genre,Budget,Platform
0,1515139200,6180.0,Horror,10000000.0,others
1,1515139200,5700.0,Action,NaN,others
2,1522998000,6780.0,Adventure,NaN,others
3,1515744000,6240.0,Action,30000000.0,others
4,1515744000,5460.0,Comedy,NaN,others
...,...,...,...,...,...
1148,1671177600,11520.0,Action,350000000.0,others
1149,1671609600,6120.0,Animation,90000000.0,others
1150,1672387200,6360.0,Documentary,NaN,others
1151,1671782400,11340.0,Comedy,78000000.0,others


In [33]:
# Create an instance of OneHotEncoder
encoder = OneHotEncoder()
genre=features['Genre']
one_hot = encoder.fit_transform(features[['Genre']])
one_hot_df = pd.DataFrame(one_hot.toarray())
df_genre=pd.concat([genre, one_hot_df], axis=1)

In [34]:
df_genre

,Genre,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Horror,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Action,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adventure,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Action,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,Action,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1149,Animation,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1150,Documentary,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1151,Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Create an instance of OneHotEncoder
encoder = OneHotEncoder()
genre=features['Platform']
one_hot = encoder.fit_transform(features[['Platform']])
one_hot_df = pd.DataFrame(one_hot.toarray())
df_one_hot = pd.concat([genre, one_hot_df], axis=1)

In [35]:
df_platform=df_one_hot

In [36]:
df_platform

,Platform,0,1,2,3,4
0,others,0.0,0.0,0.0,0.0,1.0
1,others,0.0,0.0,0.0,0.0,1.0
2,others,0.0,0.0,0.0,0.0,1.0
3,others,0.0,0.0,0.0,0.0,1.0
4,others,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
1148,others,0.0,0.0,0.0,0.0,1.0
1149,others,0.0,0.0,0.0,0.0,1.0
1150,others,0.0,0.0,0.0,0.0,1.0
1151,others,0.0,0.0,0.0,0.0,1.0


In [39]:
features['Budget'].isna().count()

1153